In [ ]:
# Import libraries
from openslide import OpenSlide
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import ndimage
from skimage import io, color, measure
import csv

In [ ]:
# Load an example SVS file
svs_path = "/Users/ashishsingh/Desktop/470650.svs" 
slide = OpenSlide(svs_path)

# Print some basic slide properties
print("Dimensions of full image (level 0):", slide.dimensions)
print("Number of levels:", slide.level_count)
print("Dimensions of each level:", [slide.level_dimensions[i] for i in range(slide.level_count)])
vendor = slide.properties.get("openslide.vendor") # Check vendor
print(vendor)

# Select the lowest resolution level (highest level number)
lowest_res_level = slide.level_count - 1

# Read the whole slide at the selected low resolution level
low_res_image = slide.read_region((0, 0), lowest_res_level, slide.level_dimensions[lowest_res_level])
low_res_image = low_res_image.convert("RGB")  # Convert to RGB for better visualization

# Display the low-resolution whole slide image
plt.figure(figsize=(10, 10))
plt.imshow(low_res_image)
plt.axis("off")
plt.show()

In [ ]:
# Display PNG thumbnail
tn = slide.get_thumbnail(size=(500, 500))
tn.show()

In [ ]:
# Manipulate thumbnail
tn_np = np.array(tn)
print(tn_np.size)
print(tn_np.shape)

plt.figure(figsize=(8,8))
plt.imshow(tn_np)

In [ ]:
# Manipulate slide

dims = slide.level_dimensions
print(dims)
print(len(dims))

factors = slide.level_downsamples
print("Each level is downsampled by", factors)

SCALE_FACTOR=32
best_level = slide.get_best_level_for_downsample(SCALE_FACTOR)
print(best_level)

level3_dim = dims[2]
level3_img = slide.read_region((0,0),2, level3_dim)
print(level3_img)

level3_img_RGB = level3_img.convert('RGB')
level3_img_RGB.show()

level3_img_np = np.array(level3_img_RGB)
plt.imshow(level3_img_np)

In [ ]:
# Manipulate tiles

from openslide.deepzoom import DeepZoomGenerator

tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)

print("Number of levels", tiles.level_count)
print("Dimensions in each level", tiles.level_dimensions)
print("Total number of tiles", tiles.tile_count)

In [ ]:
# Checking an example tile's features

level_num=11
print("The tiles shape at level", level_num, "is:", tiles.level_tiles[level_num])

print("This means there are", tiles.level_tiles[level_num][0]* tiles.level_tiles[level_num][1])

tile_dims = tiles.get_tile_dimensions(11,(0,0))
print(tile_dims)

In [ ]:
# Exploring an example tile at level 16

tile_count_in_large_image = tiles.level_tiles[16]
print(tile_count_in_large_image)

tile_dims = tiles.get_tile_dimensions(16,(111,100))
print(tile_dims)

single_tile = tiles.get_tile(16, (20,40))
single_tile_RGB = single_tile.convert('RGB')
single_tile_RGB.show()

In [ ]:
# Saving all tiles locally from an SVS file

cols, rows = tiles.level_tiles[16]

import os
tile_dir = "/Users/ashishsingh/Desktop/tiles"
for row in range (rows):
    for col in range (cols):
        tile_name = os.path.join(tile_dir, f"{col}_{row}")
        print ("Now saving tile with title: ", tile_name)
        temp_tile = tiles.get_tile(16,(col,row))
        temp_tile_RGB = temp_tile.convert('RGB')
        temp_tile_np = np.array(temp_tile_RGB)
        plt.imsave(tile_name + ".png", temp_tile_np)

In [ ]:
# Example tile grain size analysis

img = cv2.imread("/Users/ashishsingh/Desktop/60_28.png", 0)
pixels_to_um = 0.5

plt.hist(img.flat, bins=100, range=(0,255))

ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) # image thresholding

kernel = np.ones((3,3), np.uint8) # kernel for mophological operations below
eroded = cv2.erode(thresh, kernel, iterations = 1) # image erosing
dilated = cv2.dilate(eroded, kernel, iterations = 1) # image dilatation

cv2.imshow("Threshold image", thresh)
cv2.imshow("Eroded image", eroded)
cv2.imshow("Dilated image", dilated)
cv2.waitKey(0)

mask = dilated == 255 # binary masking
io.imshow(mask[150:280, 150:280])

s = [[1,1,1], [1,1,1], [1,1,1]] # structuring element for labelling connected components
labeled_mask, num_labels = ndimage.label(mask, structure=s) # labelling connected components in the binary mask

img2 = color.label2rgb(labeled_mask, bg_label=0)
cv2.imshow("Coloured labels", img2)
cv2.waitKey(0)

# Generating a csv file of regional features

propList = ['Area',
            'equivalent_diameter',
            'orientation',
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']

with open('image_measurements.csv', 'w', newline='', encoding='utf-8') as output_file:
    writer = csv.writer(output_file)
    writer.writerow(['Label'] + propList)  # Header row
    for cluster_props in clusters:
        row = [cluster_props['Label']]
        for prop in propList:
            if prop == 'Area':
                to_print = cluster_props[prop] * pixels_to_um**2
            elif prop == 'orientation':
                to_print = cluster_props[prop] * 57.2958
            elif prop.find('Intensity') < 0:
                to_print = cluster_props[prop] * pixels_to_um
            else:
                to_print = cluster_props[prop]
            row.append(to_print)
        writer.writerow(row)

In [ ]:
# Watershed Segmentation for an example image

img = cv2.imread("/Users/ashishsingh/Desktop/60_28.png", cv2.IMREAD_COLOR)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(img.shape)  # height, width, channels of an image to ensure colour vs grayscale
if len(img.shape) == 2:  # Grayscale image
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

pixels_to_um = 0.5

ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

sure_background = cv2.dilate(opening, kernel, iterations=3) # delineating background

dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3) # calculating distance transform of a binary image

ret2, sure_foreground = cv2.threshold(dist_transform, 0.2*dist_transform.max(), 255, 0) # delineating foreground

sure_foreground = np.uint8(sure_foreground)

unknown = cv2.subtract(sure_background, sure_foreground) # delineating margins

ret3, markers = cv2.connectedComponents(sure_foreground)
markers += 10
markers[unknown==255] = 0

markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 255, 255]

img2 = color.label2rgb(markers, bg_label=0)

cv2.imshow("Overlay original image", img)
cv2.imshow("Coloured image", img2)

plt.imshow(markers, cmap="jet")

cv2.imshow("Unknown", unknown)
cv2.waitKey(0)

from skimage.segmentation import clear_border #minimise border noise
opening = clear_border (opening)

# Generating a csv file of regional features

regions = measure.regionprops(markers, intensity_image=img)

propList = ['Area',
            'equivalent_diameter',
            'orientation',
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']

with open('image_measurements.csv', 'w', newline='', encoding='utf-8') as output_file:
    writer = csv.writer(output_file)
    writer.writerow(['Label'] + propList)  # Header row
    for region_props in regions:
        row = [region_props['Label']]
        for prop in propList:
            if prop == 'Area':
                to_print = region_props[prop] * pixels_to_um**2
            elif prop == 'orientation':
                to_print = region_props[prop] * 57.2958
            elif prop.find('Intensity') < 0:
                to_print = region_props[prop] * pixels_to_um
            else:
                to_print = region_props[prop]
            row.append(to_print)
        writer.writerow(row)

In [ ]:
# Watershed Segmentation for a folder of images

import glob

pixels_to_um = 0.5

propList = ['Area',
            'equivalent_diameter',
            'orientation',
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']

path = "/Users/ashishsingh/Desktop/tiles/*.png*"
for file in glob.glob(path):
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    from skimage.segmentation import clear_border
    opening = clear_border (opening)

    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3)

    ret2, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)

    unknown = cv2.subtract(sure_bg, sure_fg)

    ret3, markers = cv2.connectedComponents(sure_fg)
    markers += 10
    markers[unknown==255] = 0

    markers = cv2.watershed(img, markers)
    img[markers == -1] = [0, 255, 255]

    img2 = color.label2rgb(markers, bg_label=0)

    regions = measure.regionprops(markers, intensity_image=img)

    with open('image_measurements.csv', 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['Label'] + propList)  # Header row
        for region_props in regions:
            row = [region_props['Label']]
            for prop in propList:
                if prop == 'Area':
                    to_print = region_props[prop] * pixels_to_um**2
                elif prop == 'orientation':
                    to_print = region_props[prop] * 57.2958
                elif prop.find('Intensity') < 0:
                    to_print = region_props[prop] * pixels_to_um
                else:
                    to_print = region_props[prop]
                row.append(to_print)
            writer.writerow(row)